状态空间用卡尔曼滤波估计的简单例子
https://janelleturing.medium.com/advanced-time-series-analysis-state-space-models-and-kalman-filtering-3b7eb7157bf2
时变参数状态空间模型与R应用
https://statisticssu.github.io/STM/tutorial/statespace/statespace.html

In [2]:
import os
import math
import pandas as pd
import numpy as np
import scipy.optimize
from mpmath import mp
from scipy.linalg import det
from scipy.optimize import minimize, least_squares, rosen, rosen_der, rosen_hess, Bounds,check_grad,approx_fprime

ModuleNotFoundError: No module named 'mpmath'

In [4]:
class MLFA:
    def __init__(self,mY,iP,iQ):
        self.Y = mY
        self.p = iP
        self.q = iQ
         
        self.theta0 = self.__guess()   #初始值
        self.theta = self.theta0       #设置当前theta
    #s
    def __guess(self):
        N = self.Y.shape[0]
        beta = np.ones([N-1,1])                                                  #因为GDP的载荷因子被设为1，所以beta估计值少1
        theta = beta                                                             # beta：0到 N-2行，共N-1行
        if self.p>0:
            phi_f = np.zeros([1,self.p])
            vphi_f = phi_f.reshape(-1,1)
            theta = np.concatenate((theta,vphi_f),axis=0)                       #phi_f： N-1 到 N-1+p-1行 共p行 
        if self.q >0:
            phi_u = np.zeros([N,self.q])
            vphi_u = phi_u.reshape(-1,1) #共N*q行
            theta = np.concatenate((theta,vphi_u),axis=0)                       #phi_u： N-1+p 到 N-1+p+N*q-1 行 共N*q 行
        dsig_11 = 0.5* np.ones([1,1]) 
        vsig_22 = 0.5* np.ones([N,1])
        theta = np.row_stack((theta,dsig_11,vsig_22))                           #variance named "sig": 从 N-1+p+N*q 到 N-1+p+N*q+N行 共1+N行
        theta = theta.flatten() 
        return theta                                                            # 从0到 N-1+p+q+N 行，共 N-1+p+q+N+1 行 
    
    def dfm(self,theta,N):         
        cbeta = N - 1
        cphi_f =  self.p
        cphi_u = N *  self.q
        cphi = cphi_f + cphi_u
        ctheta = len(theta)
        
        # Ensure dimensions match when concatenating
        beta = np.row_stack((np.array([1]), theta[0:cbeta].reshape(-1, 1)))   # 提取theta的0到 cbeta-1行 共cbeta-1+1 = N-1行，再加上1这一行，beta共N行
        vphi_f = np.zeros([0,0])
        #print(vphi_f.shape)=(0,0)
        if  self.p > 0:
            vphi_f0 = theta[cbeta:cbeta + cphi_f].T                           # 提取theta 从cbeta 到 cbeta+cphi_f-1行 共 cphi_f-1+1 = p 行 
            vphi_f = np.array(vphi_f0).reshape(-1)
            #print(vphi_f.shape)
        if  self.q > 0:
            element = np.array(theta[cbeta + cphi_f:cbeta + cphi_f + N])      #提取 theta 从 cbeta+cphi_f 到 cbeta+cphi_f+N-1行 共 N-1+1 = N 行
            mphi_u = np.diagflat(element)
                
            if  self.q > 1:
                for i in range(2,  self.q + 1):
                    start_index = cbeta + cphi_f + (i - 1) * N
                    end_index = cbeta + cphi_f + i * N
                    element = theta[start_index:end_index]                     #切片theta 从 ...+N 到 ...+2N 共N+1-1=N行
                    amphi_u_emp = np.diagflat(element)
                    mphi_u = np.hstack((mphi_u, amphi_u_emp))
                    
        element = theta[cbeta + cphi]  # Ensure element is a scalar for dsig_11  # 切片theta 第cbeta+cphi+1行（因为从0开始的）
        dsig_11 = np.array([[np.abs(element)]])  # Create a 2D array for np.diag
        element = theta[cbeta + cphi + 1:ctheta]
        msig_22 = np.diagflat(element)
        return beta, vphi_f, mphi_u, dsig_11, msig_22

    def ssr(self,N,N_1,beta, vphi_f, mphi_u, dsig_11, msig_22):
        N_2 = N -N_1
        cbeta = N
        cbeta_1 = N_1
        csta = 5+5*N
        
        F = np.zeros([csta,csta])
        df = pd.DataFrame(F)
        F[1:5,0:4]=np.identity(4)
        #df.loc[1:4,0:3] = np.eye(4)                                  # row[1-4] column[0-3] (including the ending points)
        F[5+N:5+5*N,5:5+4*N]== np.identity(4*N)
        #df.loc[5+N:5+5*N-1,5:5+4*N-1] = np.identity(4*N)
        if self.p>0:
            F[0,0:self.p] =vphi_f                             # vphi_f = [phi_f,1  ... phi_f,p]
            #print(F[0,0:p].shape)
        if self.q>0:
            F[5:5+N,5:5+(self.q*N)]=mphi_u 
        #print(F)
            
        G = np.zeros([csta,1+N])  
        G[0][0]=1
        df = pd.DataFrame(G)
        df.loc[5:5+N-1,1:1+N-1]=np.identity(N)
        #print(G)
            
        H_1 = [0]
        if N_1>0:
            beta1=beta[0:cbeta_1]   # 切片包含起止节点不包含终止节点
            H_1 = np.zeros([N_1,csta])
            H_1[:,0:1]=(1/3)*beta1
            H_1[:,1:2]=(2/3)*beta1
            H_1[:,2:3]=beta1
            H_1[:,3:4]=(2/3)*beta1
            H_1[:,4:5]=(1/3)*beta1
            H_1[:,5:5+N_1]=(1/3)*np.identity(N_1)
            H_1[:,5+N:5+N+N_1]=(2/3)*np.identity(N_1)
            H_1[:,5+2*N:5+2*N+N_1]=np.identity(N_1)
            H_1[:,5+3*N:5+3*N+N_1]=(2/3)*np.identity(N_1)
            H_1[:,5+4*N:5+4*N+N_1]=(1/3)*np.identity(N_1)
            
        beta2=beta[cbeta_1:cbeta]  # cebta_1 = N_1
        H_2 = np.zeros([N_2,csta])
        H_2[:,0:1]=beta2
        H_2[:,5+N-N_2:5+N]= np.identity(N_2)
        
        msig_vv = np.identity(1+N)
        msig_vv[0][0]= dsig_11[0][0]
        msig_vv[1:1+N,1:1+N]=msig_22
        return F,G,H_1,H_2,msig_vv

    def kf(self,F,G,H_1,H_2,msig_vv):
        Y = self.Y
        N = Y.shape[0]
        N_1 =  N_1 = np.sum(np.isnan(Y[:, 0]))
        cobs = Y.shape[1]
        csta = 5 + 5*N
        
        lnl = np.zeros([cobs,1])
        index_u = np.zeros([cobs,1])
        s_u = np.zeros([csta,1])      # state space 
        p_u = np.zeros([csta,csta])   #vairance and covariance 
    
        def trans(x):
            result = np.transpose(x)
            return result
        
        for t in range(cobs):
            #prediction    
            s_p = F @ s_u  
            p_p = F @ p_u @ trans(F) + G @ msig_vv @ trans(G)          
                    
            #log-likelihood
            y0 = np.array(Y[:,t])
            y = y0.reshape(-1,1)
            y = np.matrix(y)
            H = H_2
            sig_ww = np.zeros([N,N])
            if N_1 >0:
                H = np.vstack((H_1,H_2))
                if np.any(np.isnan(y)):
                    df = pd.DataFrame(y)
                    y = df.fillna(0)
                    rows, cols = H_1.shape
                    H_zero = np.zeros([rows,cols])
                    H = np.vstack((H_zero,H_2))
                    sig_ww[0:N_1,0:N_1] = np.identity(N_1)
            
            e = y-H @ s_p                                 # 向前一步的预测误差
            sig_ee = H @ p_p @ trans(H) + sig_ww          # 向前一步预测误差的协方差矩阵
            sign, determinant = np.linalg.slogdet(sig_ee)
            if sign != 1:   
                lnl[t] = np.inf
            else:
                try:
                    inv_sig_ee = np.linalg.inv(sig_ee)
                    lnl[t] = - (N / 2) * np.log(2 * np.pi) - 0.5 * mp.det(mp.matrix(sig_ee)) - 0.5 * (e.T @ inv_sig_ee @ e)
                except np.linalg.LinAlgError:
                    lnl[t] = np.inf
                
            # updating
            gain = p_p @ trans(H) @ np.linalg.inv(H @ p_p @ trans(H) + sig_ww)  #卡尔曼增益
            s_u = s_p + gain @ (y-H @ s_p)                                      
            p_u = p_p - gain @ H @ p_p
            df = pd.DataFrame(index_u)
            df.loc[t,:] = s_u[0]  #提取更新方程中计算的f_t
        return lnl, index_u

    def loglikelihood(self,theta):
        Y = self.Y
        N = Y.shape[0]
        N_1 = np.sum(np.isnan(Y[:, 0]))
        
        beta, vphi_f, mphi_u, dsig_11, msig_22 = self.dfm(theta, N)
        F, G, H_1, H_2, msig_vv = self.ssr(N,N_1,beta, vphi_f, mphi_u, dsig_11, msig_22)
        lnl, index_u = self.kf(F,G,H_1,H_2,msig_vv)

        valid_lnl = lnl[np.isfinite(lnl)]  #isfinite
        if len(valid_lnl) ==0:
            return np.inf
        MLE = -np.sum(valid_lnl,axis=0)
        print(f"Current theta: {theta}")
        print(f"Current log-likelihood: {MLE}")
        return MLE


    def theta_fixed(self):
            theta_fixed = self.theta0
            n = self.p+self.q
            for i in range(n):
                theta_fixed[i]=0.5
            return theta_fixed
        
    def estimate(self):
        N = Y.shape[0]
        cobs = Y.shape[1]

        history = {'parameters': [], 'values': []} 
        def make_callback(history):
            def callback(theta):
                """
                Callback function to monitor optimization progress.
                theta: Current parameter values.
                """
                # 保存当前参数值
                history['parameters'].append(theta)
                
                # 计算并保存当前目标函数值
                current_value = self.loglikelihood(theta)
                history['values'].append(current_value)
                
                # 打印信息
                print(f"Iteration {len(history['values'])}: Current function value = {current_value}")
            return callback

         # 创建 callback 函数
        callback = make_callback(history)
        
        result = minimize( fun=lambda theta: self.loglikelihood(theta),
                            x0=self.theta0, 
                            method='BFGS',
                            jac = None,
                            options={'disp': True ,'maxiter': 1000,'gtol':1e-3,'eps':1e-03},
                            callback = callback
                            )
        self.theta_result = result.x
        #optimal_lnl_value = -result.fun    
        #dlnl = cobs * optimal_lnl_value
        dlnl = -result.fun
        cpar = N + self.p + N * self.q + N
        dAIC = (dlnl - cpar) / cobs
        dBIC = (dlnl - cpar * np.log(cobs) / 2) / cobs
        print(f"the value of p,q,lnl,AIC,BIC is {self.p, self.q, dlnl, dAIC, dBIC}")
        return self.theta_result
        
    def index_u(self,theta):
        
        N = Y.shape[0]
        rows_with_nan = df.isna().any(axis=1)
        N_1 = rows_with_nan.sum()
        
        beta, vphi_f, mphi_u, dsig_11, msig_22 = self.dfm(theta, N)
        F, G, H_1, H_2, msig_vv = self.ssr(N,N_1,beta, vphi_f, mphi_u, dsig_11, msig_22)
        lnl, index_u = self.kf(F,G,H_1,H_2,msig_vv)
        
        return index_u

In [7]:
data= pd.read_excel(r'F:/MixedData/LargeStateSpace/中国宏观变量数据-CEIC2005.xlsx',index_col=False,sheet_name='main2005') 
df=pd.DataFrame(data)
df.drop(labels={'Unnamed: 0'},axis=1,inplace=True)

Y0 = df.values 
Y = Y0.T

df=pd.DataFrame(Y)
N = Y.shape[0]
rows_with_nan = df.isna().any(axis=1)
N_1 = rows_with_nan.sum()
p=1
q=2

FileNotFoundError: [Errno 2] No such file or directory: 'F:/MixedData/LargeStateSpace/中国宏观变量数据-CEIC2005.xlsx'

In [3]:
mlfa = MLFA(Y,p,q)
optimal_theta = mlfa.estimate()
print(f"the optimal theta: {optimal_theta}")

NameError: name 'MLFA' is not defined